In [1]:
#Import Important Libraries
import json
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
#import piexif
import math

In [2]:
# Function to convert GPS components to decimal degrees
def convert_to_decimal_degrees(gps_component):
    degrees = gps_component[0]
    minutes = gps_component[1]
    seconds = gps_component[2]

    decimal_degrees = degrees + minutes / 60.0 + seconds / 3600.0

    return decimal_degrees

# Function to get geo-registration information from an image
def get_geo_registration(image_path):
    with Image.open(image_path) as img:
        exif_data = img._getexif()

        if exif_data is not None:
            gps_info = None
            for tag, value in exif_data.items():
                tag_name = TAGS.get(tag, tag)
                if tag_name == 'GPSInfo':
                    gps_info = value
                    #print(gps_info)
                
            if gps_info:
                latitude_degrees, latitude_minutes, latitude_seconds = gps_info[2] #latitude info extracted
                longitude_degrees, longitude_minutes, longitude_seconds = gps_info[4] #longitude info extracted
                altitude = -gps_info[6] #altitude info extracted
                

                latitude = convert_to_decimal_degrees((latitude_degrees, latitude_minutes, latitude_seconds))
                if gps_info[1] == 'S': # always negative
                    latitude = -latitude

                longitude = convert_to_decimal_degrees((longitude_degrees, longitude_minutes, longitude_seconds))
                if gps_info[3] == 'E':
                    longitude = -longitude

                # Convert altitude to a float and format as a string with 6 digits precision
                altitude = float(altitude)
                altitude_str = f"{altitude:.6f}"

                # Convert longitude and latitude (decimal degrees) to radians
                latitude_rad = math.radians(latitude)
                longitude_rad = math.radians(longitude)

                # Constants for ECEF conversion
                a = 6378137.0  # semi-major axis of the Earth (in meters)
                f = 1 / 298.257223563  # flattening factor
                e2 = 2 * f - f**2  # eccentricity squared

                # Conversion of GPS information to EPES Coordinate
                N = a / math.sqrt(1 - e2 * (math.sin(latitude_rad)**2))
                X = (N + altitude) * math.cos(latitude_rad) * math.cos(longitude_rad)
                Y = (N + altitude) * math.cos(latitude_rad) * math.sin(longitude_rad)
                Z = (N * (1 - e2) + altitude) * math.sin(latitude_rad)
                
                filename = os.path.basename(image_path)  # Extract filename without path

                geo_coordinates = f"{filename} {latitude:.6f} {longitude:.6f} {altitude_str}\n" #GPS information for each image in the JSON format
                ecef_coordinates = f"{filename}  {X:.6f} {Y:.6f} {Z:.6f}\n"  #ECEF cooridnates for each image in the JSON format

                return geo_coordinates, ecef_coordinates

## Extract the image filename from Json file 

In [3]:
# Specify the path to your JSON file
json_file_path = '/home/d8/nerfstudio/output/transforms.json'

# Create a list to store the modified file paths
modified_file_paths = []

try:
    # Open and load the JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

        # Loop through all frames and extract "file_path"
        for frame in data.get("frames", []):
            file_path = frame.get("file_path")
            if file_path is not None:
                # Remove "images/" from the file path
                modified_file_path = os.path.basename(file_path)
                modified_file_paths.append(modified_file_path)

        # Sort the file paths based on the frame number (assuming the frame number is part of the filename)
        modified_file_paths = sorted(modified_file_paths, key=lambda x: int(x.split("_")[1].split(".")[0]))

except FileNotFoundError:
    print(f"File not found: {json_file_path}")
except json.JSONDecodeError:
    print(f"Invalid JSON format in: {json_file_path}")

size = len(modified_file_paths)
print(modified_file_paths)
print(f"The size of modified_file_paths is {size}")

['frame_00001.JPG', 'frame_00002.JPG', 'frame_00003.JPG', 'frame_00004.JPG', 'frame_00005.JPG', 'frame_00006.JPG', 'frame_00007.JPG', 'frame_00008.JPG', 'frame_00009.JPG', 'frame_00010.JPG', 'frame_00011.JPG', 'frame_00012.JPG', 'frame_00013.JPG', 'frame_00014.JPG', 'frame_00015.JPG', 'frame_00016.JPG', 'frame_00017.JPG', 'frame_00018.JPG', 'frame_00019.JPG', 'frame_00020.JPG', 'frame_00021.JPG', 'frame_00022.JPG', 'frame_00023.JPG', 'frame_00024.JPG', 'frame_00025.JPG', 'frame_00026.JPG', 'frame_00027.JPG', 'frame_00028.JPG', 'frame_00029.JPG', 'frame_00030.JPG', 'frame_00031.JPG', 'frame_00032.JPG', 'frame_00033.JPG', 'frame_00034.JPG', 'frame_00035.JPG', 'frame_00036.JPG', 'frame_00037.JPG', 'frame_00038.JPG', 'frame_00039.JPG', 'frame_00040.JPG', 'frame_00041.JPG', 'frame_00042.JPG', 'frame_00043.JPG', 'frame_00044.JPG', 'frame_00045.JPG', 'frame_00046.JPG', 'frame_00047.JPG', 'frame_00048.JPG', 'frame_00049.JPG', 'frame_00050.JPG', 'frame_00051.JPG', 'frame_00052.JPG', 'frame_0005

## Check missing image frames in the Jason file (Images for which poses are not estimated)

In [4]:
image_directory = '/home/d8/3D-2D-Projection/SteelGirderImages'
all_images = [img for img in os.listdir(image_directory) if img.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
num_images_in_directory = len(all_images)
print(num_images_in_directory)

if size != num_images_in_directory:
    # Extract frame numbers from modified_file_paths
    extracted_frame_numbers = {int(x.split("_")[1].split(".")[0]) for x in modified_file_paths}
    #print(extracted_frame_numbers)

    # Find the expected range of frame numbers based on the number of images in the directory
    expected_frame_numbers = set(range(1, num_images_in_directory + 1))
    #print(expected_frame_numbers)

    # Find missing frame numbers
    missing_frame_numbers = expected_frame_numbers - extracted_frame_numbers
    
    
    if missing_frame_numbers:
        print(f"Missing frame numbers: {missing_frame_numbers}")
        print(len(missing_frame_numbers))
    else:
        print("No missing frame numbers.")
else:
    print("Number of modified_file_paths is equal to the number of images in the directory.")
    print('missing_frame_number: 0')

101
Number of modified_file_paths is equal to the number of images in the directory.
missing_frame_number: 0


## Extract the GPS and ECEF cordinates for each image frame whose pose information extracted

In [5]:
# Create and save the information for each image
geo_data = [] #it will contain the GPS information for each of the image in JSON format in the same order
ecef_data = [] # it will contain the ECEF coordinate informatin for each of the image in JSON format in the same order

# Specify the image directory
image_directory = '/home/d8/3D-2D-Projection'

# Exract the GPS and ECEF coordinates for each image as in JSON format
for modified_file_path in modified_file_paths:
    image_path = os.path.join(image_directory, 'SteelGirderImages', modified_file_path)


    if os.path.exists(image_path):
        geo_coords, ecef_coords = get_geo_registration(image_path)
        if geo_coords is not None:
            geo_data.append(geo_coords)
        if ecef_coords is not None:
            ecef_data.append(ecef_coords)
    else:
        print(f"Image not found: {image_path}")

print(geo_data)
print(len(geo_data))
print(ecef_data)
print(len(ecef_data))

['frame_00001.JPG 36.999462 -127.556562 -174.300000\n', 'frame_00002.JPG 36.999479 -127.556595 -174.148000\n', 'frame_00003.JPG 36.999484 -127.556615 -174.025000\n', 'frame_00004.JPG 36.999488 -127.556645 -173.838000\n', 'frame_00005.JPG 36.999490 -127.556697 -173.697000\n', 'frame_00006.JPG 36.999485 -127.556740 -173.630000\n', 'frame_00007.JPG 36.999476 -127.556776 -173.709000\n', 'frame_00008.JPG 36.999470 -127.556791 -173.710000\n', 'frame_00009.JPG 36.999451 -127.556821 -173.689000\n', 'frame_00010.JPG 36.999407 -127.556864 -173.655000\n', 'frame_00011.JPG 36.999381 -127.556871 -173.712000\n', 'frame_00012.JPG 36.999327 -127.556852 -173.712000\n', 'frame_00013.JPG 36.999314 -127.556828 -173.537000\n', 'frame_00014.JPG 36.999308 -127.556818 -173.491000\n', 'frame_00015.JPG 36.999300 -127.556780 -173.445000\n', 'frame_00016.JPG 36.999301 -127.556755 -173.524000\n', 'frame_00017.JPG 36.999300 -127.556732 -173.565000\n', 'frame_00018.JPG 36.999303 -127.556702 -173.790000\n', 'frame_00

## Save extracted geo-coordinates and ecef coordinates as .txt file

In [6]:
#Specify the output directory
output_directory = '/home/d8/3D-2D-Projection/Georegistratio/Coordinates_data'

# Save geo-coordinates to a text file
geo_txt_file_path = os.path.join(output_directory, 'geo_coordinates_3.txt')
with open(geo_txt_file_path, 'w') as geo_txt_file:
    geo_txt_file.writelines(geo_data)

print("Geo-coordinates saved to geo_coordinates.txt.")

# Save ECEF coordinates to a text file
ecef_txt_file_path = os.path.join(output_directory, 'ecef_coordinates_3.txt')
with open(ecef_txt_file_path, 'w') as ecef_txt_file:
    ecef_txt_file.writelines(ecef_data)

print("ECEF coordinates saved to ecef_coordinates.txt.")

Geo-coordinates saved to geo_coordinates.txt.
ECEF coordinates saved to ecef_coordinates.txt.
